In [16]:
loc.find('h2', attrs = 'pane-title').text

u'Library Locations'

In [36]:
my_list = []
for loc in all_locations:
        if loc.find('div', attrs = 'views-field-field-address') is not None:
            my_list.append(loc.find('h2', attrs = 'pane-title').text)
my_list

[u'Faulk Central Library',
 u'Austin History Center',
 u'Carver Branch',
 u'Cepeda Branch',
 u'Hampton Branch at Oak Hill',
 u'Howson Branch',
 u'Little Walnut Creek Branch',
 u'Manchaca Road Branch',
 u'Mobile Library',
 u'Milwood Branch',
 u'North Village Branch',
 u'Old Quarry Branch',
 u'Pleasant Hill Branch',
 u'Recycled Reads Bookstore',
 u'Ruiz Branch',
 u'St. John Branch',
 u'Southeast Branch',
 u'Spicewood Springs Branch',
 u'Terrazas Branch',
 u'Twin Oaks Branch',
 u'University Hills Branch',
 u'Willie Mae Kirk Branch',
 u'Windsor Park Branch',
 u'Yarborough Branch']

In [18]:
address = soup.find('div', attrs = 'views-field-field-address').text

u' 800 Guadalupe St., Austin, TX, 78701 '

In [152]:
from __future__ import print_function

import os
from ftplib import FTP
from bs4 import BeautifulSoup
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.base import TransformerMixin
from sklearn.datasets import load_files
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import TfidfTransformer
from nltk.corpus import stopwords
import string
import numpy as np
import re
import glob


In [86]:
!ls

Bill Text Train-Test Creation.ipynb document_clustering.ipynb
README.md                           tx


In [32]:
ftp = FTP('ftp.legis.state.tx.us')
ftp.login()

'230 Anonymous user logged in.'

In [13]:
#Directory structure: /bills/<legislative session>/<document type>/<text format>/<bill type>/<grouping of 100 bills>
ftp.cwd('/bills/851/billtext/html/house_bills/HB00001_HB00099')
files = ftp.nlst()
print files

['HB00001E.htm', 'HB00001H.htm', 'HB00001I.htm', 'HB00002E.htm', 'HB00002H.htm', 'HB00002I.htm', 'HB00003I.htm', 'HB00004H.htm', 'HB00004I.htm', 'HB00005I.htm', 'HB00006I.htm', 'HB00007E.htm', 'HB00007F.htm', 'HB00007H.htm', 'HB00007I.htm', 'HB00007S.htm', 'HB00009E.htm', 'HB00009H.htm', 'HB00009I.htm', 'HB00010E.htm', 'HB00010H.htm', 'HB00010I.htm', 'HB00011E.htm', 'HB00011H.htm', 'HB00011I.htm', 'HB00012I.htm', 'HB00013E.htm', 'HB00013F.htm', 'HB00013H.htm', 'HB00013I.htm', 'HB00013S.htm', 'HB00014H.htm', 'HB00014I.htm', 'HB00015H.htm', 'HB00015I.htm', 'HB00016H.htm', 'HB00016I.htm', 'HB00017H.htm', 'HB00017I.htm', 'HB00018H.htm', 'HB00018I.htm', 'HB00019H.htm', 'HB00019I.htm', 'HB00020E.htm', 'HB00020H.htm', 'HB00020I.htm', 'HB00021E.htm', 'HB00021F.htm', 'HB00021H.htm', 'HB00021I.htm', 'HB00021S.htm', 'HB00022H.htm', 'HB00022I.htm', 'HB00023E.htm', 'HB00023H.htm', 'HB00023I.htm', 'HB00024H.htm', 'HB00024I.htm', 'HB00025E.htm', 'HB00025H.htm', 'HB00025I.htm', 'HB00026E.htm', 'HB0002

## Download html files from FTP

In [88]:
# The naming convention for individual documents is: <bill chamber><bill type><bill suffix><bill version>
# File Name	Description
# <bill chamber>	H=House, S=Senate
# <bill type>	B=Bill, J=Joint Resolution, R=Resolution, C=Concurrent Resolution
# <bill suffix>	5-digit number
# <bill version>	I=Introduced, S=Senate Committee Report, H=House Committee Report, E=Engrossed, F=Enrolled


# def is_empty(file):
#     file_size=os.stat(file).st_size
#     if file_size == 0:
#         print "The file {} is empty".format(file)
#         return True
#     return False

for file in files:
    try:
        local_filename = os.path.join(local_dir, file)
        print "Getting filename " + file
        ftp.retrbinary('RETR %s' % file, open(local_filename, 'wb').write)
        print "Saving at %s" % local_filename
    except Exception, err:
        print err
#         if (is_empty(local_filename)):
#             os.remove(local_filename)
        continue

Getting filename HB00001E.htm
[Errno 49] Can't assign requested address
Getting filename HB00001H.htm
[Errno 32] Broken pipe
Getting filename HB00001I.htm
[Errno 32] Broken pipe
Getting filename HB00002E.htm
[Errno 32] Broken pipe
Getting filename HB00002H.htm
[Errno 32] Broken pipe
Getting filename HB00002I.htm
[Errno 32] Broken pipe
Getting filename HB00003I.htm
[Errno 32] Broken pipe
Getting filename HB00004H.htm
[Errno 32] Broken pipe
Getting filename HB00004I.htm
[Errno 32] Broken pipe
Getting filename HB00005I.htm
[Errno 32] Broken pipe
Getting filename HB00006I.htm
[Errno 32] Broken pipe
Getting filename HB00007E.htm
[Errno 32] Broken pipe
Getting filename HB00007F.htm
[Errno 32] Broken pipe
Getting filename HB00007H.htm
[Errno 32] Broken pipe
Getting filename HB00007I.htm
[Errno 32] Broken pipe
Getting filename HB00007S.htm
[Errno 32] Broken pipe
Getting filename HB00009E.htm
[Errno 32] Broken pipe
Getting filename HB00009H.htm
[Errno 32] Broken pipe
Getting filename HB00009I.h

In [190]:
local_dir = '/Users/rhaggerty/remote/bills/tx/data/'

In [180]:
output_dir

'/Users/rhaggerty/remote/bills/tx/data/text'

In [187]:
def prettify_soup(doc_name):
#     'HB00098I'
    bill_no = doc_name.split('/')[-1].rstrip('.htm')
    output_dir = os.path.join(local_dir, 'text/')
    with open(doc_name) as in_file:
        soup = BeautifulSoup(in_file, 'html.parser') 
        pretty_text = soup.get_text(' ', strip=True)
        with open(output_dir + bill_no + '.txt', 'w+') as out_file:
            out_file.write(pretty_text.encode('utf8'))

In [189]:
for doc_name in glob.glob(local_dir + 'html/*.htm'):
    #doc_name = '/Users/rhaggerty/remote/bills/tx/data/html/HB00098I.htm'
    prettify_soup(doc_name)

In [171]:
!ls /Users/rhaggerty/remote/bills/tx/data/text

HB00098I.txt


In [193]:

# if ends in period, then:
# summary = soup.find(text=re.compile('relating'))
# summary
# else get next line and append
# summary.next_element.next_element.next_element.get_text()


summary_list = []
summary_dict = {}
for doc_name in glob.glob(local_dir + 'html/*.htm'):
    bill_no = doc_name.split('/')[-1].rstrip('.htm')
    with open(doc_name) as in_file:
        soup = BeautifulSoup(in_file, 'html.parser')
        summary = soup.find(text=re.compile('relating'))
    summary_list.append(summary)
print(summary_list)


[u'relating to avoiding the abolishment of certain agencies subject to ', u'relating to avoiding the abolishment of certain agencies subject to ', u'relating to avoiding the abolishment of certain agencies subject to ', u'relating to the repeal of certain riders for the Texas Medical Board ', u'relating to the repeal of certain riders for the Texas Medical Board ', u'relating to the funding of certain governmental entities subject to ', u'relating to ad valorem taxation; authorizing fees.', u'relating to the calculation of the ad valorem rollback tax rate of a ', u'relating to the calculation of the ad valorem rollback tax rate of a ', u'relating to a limitation on increases in the appraised value for ad ', u'relating to municipal annexation.', u'relating to a tree planting credit to offset tree mitigation fees ', u'relating to municipal regulation of the removal of trees on private ', None, u'relating to a tree planting credit to offset tree mitigation fees ', u'relating to municipal 

In [198]:
summary_list = []
bill_no_list = []
for doc_name in glob.glob(local_dir + 'html/*.htm'):
    bill_no = doc_name.split('/')[-1].rstrip('.htm')
    with open(doc_name) as in_file:
        soup = BeautifulSoup(in_file, 'html.parser')
        summary = soup.find(text=re.compile('relating'))
    summary_list.append(summary)
    bill_no_list.append(bill_no)
bill_summaries = zip(bill_no_list, summary_list)
for bill, summary in bill_summaries:
    summary_dict[bill] = summary
print(summary_dict)

{'HB00010I': u'relating to maternal morbidity and pregnancy-related deaths.', 'HB00010H': u'relating to maternal morbidity and pregnancy-related deaths.', 'HB00074E': u'relating to the applicability of the law governing the provision of ', 'HB00021F': u'relating to public school finance, including funding for the ', 'HB00021E': u'relating to the public school finance system.', 'HB00074I': u'relating to the applicability of the law governing the provision of ', 'HB00074H': u'relating to the applicability of the law governing the provision of ', 'HB00021I': u'relating to the public school finance system.', 'HB00021H': u'relating to the public school finance system.', 'HB00096I': u'relating to signature verification on an early voting ballot voted ', 'HB00096H': u'relating to signature verification on an early voting ballot voted ', 'HB00010E': u'relating to maternal morbidity and pregnancy-related deaths.', 'HB00005I': u'relating to a limitation on increases in the appraised value for ad

In [231]:
clean_summary = []
for summ in summary_list:
    if summ is not None:
        clean = re.sub(r'relating to', ' ', summ)
        words = clean.split()
        non_stop = [w for w in words if not w in stopwords.words("english")]
    clean_summary.append(non_stop)
#     print(clean)
clean_summary
# clean_summary

[[u'avoiding', u'abolishment', u'certain', u'agencies', u'subject'],
 [u'avoiding', u'abolishment', u'certain', u'agencies', u'subject'],
 [u'avoiding', u'abolishment', u'certain', u'agencies', u'subject'],
 [u'repeal', u'certain', u'riders', u'Texas', u'Medical', u'Board'],
 [u'repeal', u'certain', u'riders', u'Texas', u'Medical', u'Board'],
 [u'funding', u'certain', u'governmental', u'entities', u'subject'],
 [u'ad', u'valorem', u'taxation;', u'authorizing', u'fees.'],
 [u'calculation', u'ad', u'valorem', u'rollback', u'tax', u'rate'],
 [u'calculation', u'ad', u'valorem', u'rollback', u'tax', u'rate'],
 [u'limitation', u'increases', u'appraised', u'value', u'ad'],
 [u'municipal', u'annexation.'],
 [u'tree', u'planting', u'credit', u'offset', u'tree', u'mitigation', u'fees'],
 [u'municipal', u'regulation', u'removal', u'trees', u'private'],
 [u'municipal', u'regulation', u'removal', u'trees', u'private'],
 [u'tree', u'planting', u'credit', u'offset', u'tree', u'mitigation', u'fees'],


In [223]:

words = [w for w in words if not w in stopwords.words("english")]
print words

[u'  avoiding the abolishment of certain agencies subject to ',
 u'  avoiding the abolishment of certain agencies subject to ',
 u'  avoiding the abolishment of certain agencies subject to ',
 u'  the repeal of certain riders for the Texas Medical Board ',
 u'  the repeal of certain riders for the Texas Medical Board ',
 u'  the funding of certain governmental entities subject to ',
 u'  ad valorem taxation; authorizing fees.',
 u'  the calculation of the ad valorem rollback tax rate of a ',
 u'  the calculation of the ad valorem rollback tax rate of a ',
 u'  a limitation on increases in the appraised value for ad ',
 u'  municipal annexation.',
 u'  a tree planting credit to offset tree mitigation fees ',
 u'  municipal regulation of the removal of trees on private ',
 u'  municipal regulation of the removal of trees on private ',
 u'  a tree planting credit to offset tree mitigation fees ',
 u'  municipal regulation of the removal of trees or ',
 u'  maternal health and safety, preg

In [112]:
pretty_text = soup.get_text(" ", strip=True)
pretty_text

u'85(1) HB 98 - Introduced version - Bill Text td { font-family: Courier, Arial, sans-serif; font-size: 10pt; } table { empty-cells:show; } 85S10371 MK-D By:\xa0Bernal H.B.\xa0No.\xa098 A BILL TO BE ENTITLED AN ACT relating to a school district assigning a mentor teacher to a new classroom teacher. BE IT ENACTED BY THE LEGISLATURE OF THE STATE OF TEXAS: SECTION\xa01.\xa0\xa0Section 21.458, Education Code, is amended by adding Subsections (a-1), (b-1), (f), and (g) and amending Subsections (b) and (d) to read as follows: (a-1) To be assigned as a mentor, a teacher must agree to serve as a mentor teacher for at least two school years. The assignment must begin on the first day of employment of the classroom teacher to whom the mentor teacher is assigned. (b)\xa0\xa0The commissioner shall adopt rules necessary to administer this section, including rules concerning the duties and qualifications of a teacher who serves as a mentor and the number of classroom teachers that may be assigned to

In [220]:
for k,v in summary_dict.items(): 
    if summary_dict[k] is None:
        print(k) 


HB00099I
HB00007H


In [37]:
train = load_files("data/")

In [38]:
train.filenames

array([], dtype=float64)

In [39]:
train

{'DESCR': None,
 'data': [],
 'filenames': array([], dtype=float64),
 'target': array([], dtype=float64),
 'target_names': []}

In [ ]:
print("\n".join(train.data[0].split("\n")[:3]))

In [ ]:
print(train.target_names[train.target[0]])